In [224]:
from pyexpat import features
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

In [225]:
# read the data
results = pd.read_csv("data/results.csv")
premier_league = pd.read_csv("premier_data/premier_matches.csv")


C:\Users\kumars\AppData\Local\Temp\ipykernel_3792\1167503236.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  premier_league = pd.read_csv("premier_data/premier_matches.csv")


In [226]:
# dropping unusable columns
drop_cols = ["C_HTB", "MaxOver25", "MaxUnder25", "HandiSize", "HandiHome",
    "HandiAway", "C_LTH", "C_LTA", "C_VHD", "C_VAD", "C_PHB",
	"MaxHome",	"MaxDraw",	"MaxAway",	"Over25",	"Under25","MatchTime"]
premier_league = premier_league.drop(columns= drop_cols)

In [227]:
# copies only premier league data, and extracts data from 2005 to 2024
premier_league = premier_league[premier_league['Division'] == 'E0'].copy()
premier_league['MatchDate'] = pd.to_datetime(premier_league['MatchDate']).dt.year
premier_league = premier_league[premier_league['MatchDate'] >= 2005]
premier_league = premier_league[premier_league['MatchDate'] <= 2024]

In [228]:
premier_league = premier_league[premier_league['Division'] == 'E0']

In [229]:
# removing draws and mapping home,away
premier_league = premier_league[premier_league['FTResult'] != 'D'].copy()
premier_league['FTResult'] = premier_league['FTResult'].map({'H':0,'A':1})

In [230]:
# using betting odds
# normalizing probabilities
premier_league['HomeProb_raw'] = 1 / premier_league['OddHome']
premier_league['AwayProb_raw'] = 1 / premier_league['OddAway']

# makes HomeProb + AwayProb = 1
prob_sum = (premier_league['HomeProb_raw'] + premier_league['AwayProb_raw'])
premier_league['HomeProb'] = premier_league['HomeProb_raw'] / prob_sum
premier_league['AwayProb'] = premier_league['AwayProb_raw'] / prob_sum

# who is more likely to win
premier_league['Odd_Diff'] = premier_league['OddHome'] - premier_league['OddAway']
premier_league['Fav'] = np.select(
    [premier_league['OddHome'] < premier_league['OddAway'],
     premier_league['OddHome'] > premier_league['OddAway']
     ],
    ['Home','Away'],
    default='Even'
)

In [231]:
#adding more value to data, differences between ratings help provide more information to the model, create features that represent relative strength and momentum
premier_league["EloDiff"]   = premier_league["HomeElo"] - premier_league["AwayElo"]
premier_league["Form3Diff"] = premier_league["Form3Home"] - premier_league["Form3Away"]
premier_league["Form5Diff"] = premier_league["Form5Home"] - premier_league["Form5Away"]

In [232]:
# Replaces NaN values in HomeElo and AwayElo with the median computed, scale the data
away_median = x_train['AwayElo'].median()
x_train.loc[:, 'AwayElo'] = x_train['AwayElo'].fillna(away_median)
x_test.loc[:, 'AwayElo']  = x_test['AwayElo'].fillna(away_median)
x_train['HomeElo'] = x_train['HomeElo'].fillna(x_train['HomeElo'].median())
x_test['HomeElo']  = x_test['HomeElo'].fillna(x_train['HomeElo'].median())
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)

C:\Users\kumars\AppData\Local\Temp\ipykernel_3792\2709336718.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train['HomeElo'] = x_train['HomeElo'].fillna(x_train['HomeElo'].median())
C:\Users\kumars\AppData\Local\Temp\ipykernel_3792\2709336718.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test['HomeElo']  = x_test['HomeElo'].fillna(x_train['HomeElo'].median())


In [233]:
train, test = train_test_split(premier_league, test_size=0.2, random_state=0)

features = ['HomeElo',	'AwayElo','Form3Home','Form3Away','Form5Home','Form5Away','OddHome','OddDraw','OddAway','EloDiff','Form3Diff','Form5Diff','HomeProb','AwayProb']
x_train = train[features]
y_train = train['FTResult']
x_test = test[features]
y_test = test['FTResult']

pipe = Pipeline([('imputer',SimpleImputer(strategy='median')),
                 ('scaler',StandardScaler()),
                 ('lr',LogisticRegression(max_iter=1000,random_state=45))])
pipe.fit(x_train,y_train)
y_pred = pipe.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.7349081364829396
